# Technical Report
## Script Trek: NLP - Syntactic Parsing and Semantic Modeling of TV Scripts Using Natural Language Processing

by Matt Tranquada

## 1) Introduction
This notebook contains the technical writeup for my capstone project for the General Assembly Data Science Immersive program. For this project, I decided to pick a topic where I could apply Natural Language Processing, or NLP, techniques because I have a strong interest in applying NLP for semantic and discursive analysis of texts.

After reviewing a variety of potential text datasets such as news and scientific articles I became interested in analyzing film scripts for several reasons:
* As a practical working document (a "script", just like we use in computer programming), film scripts embed multiple streams of information regarding the configuration of action, dialogue, and location to be implemented on screen that can be modeled in depth.
* Film scripts structure different kinds of information using formatting patterns that are consistent but can vary across documents, making structural parsing an interesting problem.
* Film scripts describe a linear narrative structure with complex interactions and relationships between entities that can be modeled.

Originally I planned on analyzing a corpus of over 1,000 draft film scripts obtained from the Internet Screenplay Movie Database (http://www.imsdb.com), but after reviewing the data it became clear that modelling such a large sample of scripts would be too challenging in the time available because of 1) variances in the way different screenwriters format their manuscripts and use punctuation for syntax and 2) encoding problems with the script texts due to how they were presented on the IMSDb website.

Rather than abandon my idea of analyzing scripts, I decided to refocus on a sample of scripts from a single TV series to solve some of the problems and allow me to develop proof-of-concept approaches for syntactic and semantic parsing of scripts that can be reapplied and developed further for film scripts. Using scripts from a single TV show reduces variance issues because TV productions tend to standardize around a single "institutional" model for how scripts are written that is consistent across the run of the series, even if writers on individual episodes changes. The consistent format makes it much more efficient for transferring information between different production departments regardless of the story used for each episode, whereas on a film the production crew can adapt to how information is encoded in a single script that will be used for a much longer period of production.

Given the fact that it will be the 30th anniversary of the series' debut on TV this year and my personal familiarity with the series, I decided to use the scripts from the TV show "Star Trek: The Next Generation" for my analysis. What follows is a discussion of my approach and methods implemented for parsing the "Star Trek: TNG" script dataset, extracting and modeling semantic features using NLP techniques, and creating a content-based episode recommender system based on narrative analysis.

## 2) Data Sources
The primary data source for this project consisted of 176 TV scripts representing all 178 episodes from the 7 seasons of "Star Trek: The Next Generation" obtained from the Star Trek Minutiae website (http://www.st-minutiae.com/) in March 2018. Legally the text of the scripts are copyrighted by Paramount Pictures and "Star Trek" and its related trademarks are the property of CBS, but it is commonly understood that the use of script texts for educational analysis are a valid exception to copyright under the "fair use" provisions of the DMCA, provided that such use is strictly non-commercial.

The 176 script files were downloaded as plain text (.txt) files encoded in the "latin-1" text encoding format.

In addition to the script files, information from the Internet Movie Database (http://www.imdb.com) on the TV-version of each episode (such as air date, actors, etc.) were integrated into the parsed script data for cross-verification purposes and analytical enrichment.

## 3) Methodology

The goal of this project was to take the raw script text files and analyze them so they can be automatically processed in a way that allows for the extraction and comparison of key semantic features such as semantic entities, narrative structures, and dialogue patterns. In addition, I wanted to explore and validate approaches to structural and semantic analysis of other kinds of text files in the future, so I spent a lot of effort examining approaches to document strctural analysis using both supervised and unsupervised techniques, but in the end I only implemented supervised models for stryctural analysis due to time constraints.

In terms of methodology, parsing the script files to access and utilize the semantic content consists of three interconnected modeling tasks:
1. **Text file to script conversion** - The text file must be parsed to regroup the raw text "lines" into "script units" that combine semantic information with production context. This is a structural modeling problem, where the rules describing the function of each line must be inferred from some sample of the raw text files.
2. **Script to story conversion** - Once the script is represented as "script units", we can separate the different parts relevant to analyzing a tv episode, such as characters, settings, dialogue sequences, etc. While most of the parsing for this task builds on features extracted as part of the previous text file to script conversion step, I am also adding Natural Language Processing (NLP) models that are included in the Python Natural Language Toolkit (NLTK) to extract and tag important semantic features. These features allows us to look at the deeper "story vectors" that characterize different episodes and compare them in various ways.
3. **Content-based recommendation system** - Once the scripts have been enriched to make their semantic features more easily accessible, I implemented a content-based recommender system that allows a user to retrieve a list of recommended episodes with similar story characteristics to a given episode. The recommender takes into account a variety of contextually-relevant characteristics such as dominant characters in the episode, appearance of specific alien races, and story keywords to identify similar episodes. 

### 3A) Text File Analysis and Script Parsing Using Classification Models

To classify the different line types for parsing, I developed a system of 11 classification types mostly based on content and line function based on a single episode script file. The 11 categories are:
1. **Action ("A")** - A line that describes a camera or character action to be performed on screen
2. **Blank ("B")** - A line that contains only whitespace characters in addition to a newline character
3. **Character ("C")** - A line specifying the character intended to execute a line of action or dialogue
4. **Dialogue ("D")** - A line indicating text to be spoken by a character
5. **Empty ("E")** - A line containing no information except a single newline character
6. **Header ("H")** - A line containing page header information
7. **Modifier ("M")** - A line containing information meant to modify how a character performs a line of action or dialogue
8. **Note ("N")** - A line containing a note about the story metastructure or other production-side information that will not be evident in the final TV product
9. **Outro Action ("O")** - A line indicating a camera action occuring at the end of a scene
10. **Scene ("S")** - A line indicating the setting and location of the current scene
11. **Miscellaneous ("X")** - A line containing miscellaneous text not necessary for parsing story content

Because of the nature of how scripts are formatted for production, inferring the function of a specific line and the information it contains is essentially a context-free grammar (CFG) problem. In other words, there is a specific grammar or syntax applied to the structure of the script that allows us to determine the function of each line in relation to each other. Using a classification model to identify the rules for this structural grammar, it is then possible to transform the script to access the semantic content contained within each type of functional element.

In order to automate the process of parsing the script text files, my goal was to develop a supervised training model that could use training data from a single episode script to model and classify the entire set of scripts effectively. Thankfully, given that each script contains over 3000 lines on average, a single script includes 100s of examples of most kinds of line functions and at least a few examples of even the lowest frequency line functions. For example, for the training sample for season 3 episode 1, the script file contained:

_Function_ | _Count_

    A     390
    B     8
    C     425
    D     1026
    E     870
    H     71
    M     83
    N     13
    O     26
    S     128
    X     65
    
With the exception of line functions B and N, there is more than enough information in a single script to infer the rules of the structural functional grammar. In order to model the lines for classification, we must extract line-level features that can help to differentiate them. To do so, I extracted a variety of metrics related to character counts, punctuation composition, and token counts to see what the best characteristics for differentiating each type of line function would be. These metrics were provided as both raw counts (i.e. "number of commas") as well as proportionally (i.e. "number of commas / total characters") to see which would perform the best. 

I also used basic character frequency analysis of all the script text files in aggregate to detect characters that could be of syntactic significance in this context and decide which punctuation characters to extract as features. This helped to identify a number of potentially important characters to examine, including the tab character ("\t" or ASCII decimal character code 9), which turns out to be very important for indicating line function.

Once the line-level features had been extracted, I modeled the data using two techniques: random forest classification trees to help determine the most important line-level features for determining function, and logistic regression to find models specific to each line that can be used as a rule for script parsing and transformation.    

#### Random Forest Modeling of Line Functions

For my first model, I tested 27 line-level features in a 100-tree random forest to examine relative feature importance. The model achieved a score of 99.97% fitting on the single training sample, and returned the following feature importances:

Feature | Importance

    chars = 0.06938
    max_char = 0.08317
    min_char = 0.03986
    n_apostrophe = 0.00060
    n_colon = 0.00241
    n_comma = 0.00003
    n_exotics = 0.00000
    n_hyphen = 0.00059
    n_letters = 0.05363
    n_lowers = 0.03304
    n_nonletters = 0.05312
    n_nonwhitespaces = 0.04917
    n_nums = 0.00736
    n_parenthesis = 0.01987
    n_period = 0.00445
    n_quote = 0.00315
    n_slash = 0.00295
    n_space = 0.02642
    n_tab = 0.17200
    n_tokens = 0.04299
    n_uppers = 0.02721
    n_whitespaces = 0.06747
    p_lowers = 0.03805
    p_nums = 0.01120
    p_space = 0.05627
    p_tab = 0.09622
    p_uppers = 0.03937
    
Based on this, we can see that `n_tab` (17.2%), `p_tab` (9.6%), `max_char` (8.3%), `chars` (6.9%), and `n_whitespaces` (6.7%) are the most important features identified using rando forest.

To examine this further, I ran a second random forest model dropping all features with importances of less than 1%, which yielded a score of 99.97% again fitting on the single training sample and returned the following feature importances:

Feature | Importance

    chars = 0.09195
    max_char = 0.11560
    min_char = 0.02306
    n_letters = 0.04314
    n_lowers = 0.02552
    n_nonletters = 0.05018
    n_nonwhitespaces = 0.04865
    n_parenthesis = 0.02101
    n_space = 0.02502
    n_tab = 0.14191
    n_tokens = 0.03676
    n_uppers = 0.02500
    n_whitespaces = 0.04865
    p_lowers = 0.04999
    p_nums = 0.01222
    p_space = 0.06633
    p_tab = 0.12467
    p_uppers = 0.03520
    
This second model shows some slight differences among the most important features, beginning with `n_tab` (14.2%), `p_tab` (12.5%), `max_char` (11.6%), `chars` (9.2%), and `p_space` (6.6%). 

Based on these models, I decided to engineer some additional features from `n_tab` to see if that could improve modeling. I created 6 binary (dummy) variables for each of the unique values and then dropped the "0 tabs" category. Then I reran the second model including the new dummy categories to see if they help with classification compared to the simple relative comparison. The random forest model again provided a 99.97% fit to the training sample, and returned the following feature importances:

Feature | Importance

    chars = 0.08910
    max_char = 0.07815
    min_char = 0.03092
    n_letters = 0.05678
    n_lowers = 0.01597
    n_nonletters = 0.03505
    n_nonwhitespaces = 0.03839
    n_parenthesis = 0.00686
    n_space = 0.01865
    n_tab = 0.08655
    n_tokens = 0.04575
    n_uppers = 0.01375
    n_whitespaces = 0.04398
    p_lowers = 0.04303
    p_nums = 0.0107
    p_space = 0.0321
    p_tab = 0.06975
    p_uppers = 0.02558
    tab1 = 0.05281
    tab3 = 0.13862
    tab4 = 0.01466
    tab5 = 0.05097
    tab9 = 0.00179
    
Even with `n_tab` still included, the binary tab variables still account for over 25% of feature importance in the new model, indicating that they should be quite useful for logistic modeling. 

#### Logistic Regression Modeling of Line Functions

### 3B) Semantic Parsing and Modeling of Episode Scripts

Once the individual lines of text from the original script text file have been tagged, the script can be transformed into "script units", or units of action involving the setting, camera movement, or action to be portrayed on screen, that allow for deeper semantic analysis. This transformation is necessary because the information necessary for identifying each discrete script unit may be distributed across one or more types of structural units. 

For example, consider the following passage:

    116                    DATA
    117            Captain, Doctor, I am honored by
    118            your presence, but may I suggest
    119            you attend the second concert.

In the context of the original text file, this is made up of four lines or four structural units, but it only represents a single script unit within the context of the story being presented on screen. For semantic analysis, this script unit for a piece of dialogue is better represented as an object storing multiple attributes, such as:

    {'type':'dialogue',
    'character':'DATA',
    'content':'Captain, Doctor, I am honored by your presence, but may I suggest you attend the second concert.',
    'lines':[116,117,118,119]}
    
This retains all of the original structural information but allows us to access only elements we need for a given type of analysis. in addition to the `dialogue` type of script unit, there are two other kinds of script units: an `action` unit (for actions executed by characters on screen) and a `camera` unit (for actions and movements related to the camera frame). The script units are collected together within a `scene` unit that includes the setting inormation for the current scene and indexed based on the sequence the unit occurs within the scene. The scenes are themselves grouped into `act` units when possible based on annotations in the script.

In addition to transforming the script into semantic units for NLP analysis and modeling, the beginning of each script file also contains information useful for error checking and file parsing, specifically two sections containing a list of characters in the episode and a list of sets/locations in the episode. Extracting this information provides a useful means of error checking to ensure that the semantic content has been properly transformed and that, for semantic extraction for the recommender system, that all relevant entities have been correctly identified by the entity extraction routines.

The general workflow for semantic parsing involves:
1. Dividing the script file into `script` and `intro` sections
2. Extract entity lists and metadata from `intro` section
3. Process `script` section sequentially, recusrively extracting and grouping script units based on line function and sequential order

Once this process is complete, the output is contained in two parts: a dictionary containing the info extracted from the `intro` section, and a Pandas DataFrame containing the `script` section as individual scipt units.

### 3C) Creating a Content-Based Episode Recommender System

For the final part of this project, I attempted to create a content-based recommender system that is capable of recommending episodes similar to a given episode using a cosine similarity function to identify episodes with similar types of narrative properties. I attempted to create a variety of narrative features using NLP methods to extract either measurement vectors or binary indicators at the episode-level, such as:

* Relative # of lines of dialogue of main characters (Character dominance metric)
* Count of keywords such as particle names or Star Trek tech terms relative to corpus average (Subject dominance metric)
* References to alien species
* Appearance of specific locations

Ultimately there were numerous issues with this approach that limited theutility of the recommendations obtained. The biggest issue was the need to define corpus-specific vocabulary lists to capture certain narrative features. Adding or excluding a single item as part of a category can significantly change the recomendation results fairly quickly. Also, the unique science-fiction terms used in Star Trek such as the names of alien species can pose problems for lemmatization and stemming algorithms in some cases. Finally, this narrative-element-based approach seems highly subjective compared to typical recommender systems based on user ratings because of the different types of conjunctions of narrative elements that can occur and nfluence the cosine similarity calculation. Finding appropriate weights for different types of elements or performing multiple seprate cosine similarity calcuations for different narrative dimensions could provide a better quality result.

One possible alternative to this approach would be creating a recommender system based on a Naive Bayes model, which generally performs well for text classification and may not require the same degree of specification of corpus-specific vocabulary lists. If we simply care about similar words appearing in similar patterns or frequencies, a Naive Bayes approach could provide interesting results with limited corpus-specific vocabulary customization.

Another possibility is the use of unsupervised technqiues such as Latent Dirichlet Analysis (LDA) and hierarchical agglomerative clustering or other techniques such as Primary Component Analysis (PCA) to find the primary similarities and differences between episodes and redesigning the recommendation features used with those in mind. If we can collapse narrative features into common narrative groupings and create scores in relation to those specific groupings, it could achieve better performance while continuing to use the cosine similarity function.

## 4) Discussion and Further Applications

Overall, the methods applied to structural analysis and semantic parsing worked well and show good potential for application to other similar problems. However, my approach to the recommender system proved to be poorly suited to finding similar groupings of narrative features.

With regards to applying these methods to general film scripts, it seems as though successful modeling should be possible using either a portion of the script information for training or more complex techniques using character and token frequency analysis combined with a TF-IDF vectorizer. Structural analysis of a single script document without other examples that use the same exact grammar (which is the case with the Star Trek episode scripts) is difficult and poses new problems, but if there are commonalities in the underlying grammar across the documents (such as similar numbers of functional elements like those I defined in Step 3A across scripts) there may be other methods such as semi-supervised or joint training to help analyze them efficiently. It is even possible that other techniques, such as converting the text into an image array and processing it using a 1D or 2D convolutional neural network (CNN), could avoid some of the technical challenges with text analysis altogether.

For structural analysis of other kinds of documents, moving from a context-free grammar (CFG) approach for structural analysis to a probabilistic context-free grammar (PCFG) approach could have big advantages. By using frequency analysis of characters or tokens contained in a corpus of documents, it may be possible to infer the grammatical syntax of the corpus using unsupervised approaches that require nominal knowledge of the underlying documents. For example, if you imagine analyzing a large number of Python .py files, you couldlikely differentiate different kinds of functional statements based on the presence of arithmetic operators like +, -, =, etc. In the context of written documents, analysis of keyword tokens could provide similarly important clues to the structure and function of data within an unknown document.